# mf-rack
## Installing to a Fabric rack

This notebook assumes that the "ansible control node" has been setup following the instructions found at https://fabric-testbed.atlassian.net/wiki/spaces/FP/pages/376930350/MF+Prometheus+Deployment#Ansible-control-node

This notebook can be run on that node.

This notebook can be used for installation to any fabric rack, however it should be mainly used for testing. The fabric operations will handle the installations to production racks.

Note: values enclosed with greater than and less than brackets, ```<sometext>```,  need to be replaced to suit your setup.

# Set the vault password file location
The installation requires ansible hosts and variable files found in the https://github.com/fabric-testbed/fabric-deployment repository.
To access the encrpyted ansible vaults you must use the vault password found in 1Password. You can place the password in a text file to be recalled later. You may also use ``` --ask-vault-pass``` later to prevent having the password saved in a file.

In [ ]:
export VAULT_PASSWORD_FILENAME=</your_monitoring_deploy_vault_password_filnamee>

# Activate python virtual environment
It is good to create a virtual python environment for running this note book. You may use whatever python virtual enviroment tool you would like. 
Here I am using conda.


In [ ]:
conda activate <your_mf_virtual_env_name>

# Set Working Directory
The mf-rack github repository needs to be cloned to the ansible control node. fabric-deployment and mf-parsers will also be used to add some files to the cloned repository.
cd into the directory.

In [ ]:

export MF_WORKING_DIR=<the directory where the mf-rack, fabric-deployment and mf-parsers repositories will be cloned>
mkdir -p $MF_WORKING_DIR
cd $MF_WORKING_DIR

# Clone Deployment Repo

In [ ]:
cd $MF_WORKING_DIR
git clone git@github.com:fabric-testbed/mf-rack.git

# Switch Working Dir to Ansible

In [ ]:
cd mf-rack/instrumentize/ansible

# Switch to Desired Branch

In [ ]:
export BRANCH=<release_branch being deployed>
git checkout $BRANCH

In [ ]:
git pull

# Get Hosts & Vars from deployment repo
This step clones the fabric-deployent repo and copies the needed files in to the mf-rack directory

In [ ]:
export FABRIC_DEPLOYMENT_BRANCH=public-metric-additions
ansible-playbook playbook_get_fabric_deployment_files.yml --extra-vars "fabric_deployment_repo_version=$FABRIC_DEPLOYMENT_BRANCH"

# Create Ansible Config File with Vault Info
If you are entering the password for each run, delete the ```vault_password_file = $VAULT_PASSWORD_FILENAME``` text in the configuration file creation below (be sure to leave the ending quotation mark.

In [ ]:

if [[ -f "ansible.cfg" ]]; then
    echo "Config File Already Exists"
else

    echo "[defaults] 
host_key_checking = False
deprecation_warnings=False  
callbacks_enabled = profile_tasks
vault_password_file = $VAULT_PASSWORD_FILENAME" \
> ansible.cfg


echo "Created Config File"


fi

## Set Rack To Install
Set the name of the rack to install. The racks renc, uky and lbnl are currently the dev racks.

In [ ]:
export RACK_TO_INSTALL=<rack sitename>

# Install
The first time the install script is run, it will clone the mf-parsers github repository and copy the needed configuration files to the cloned mf-rack directory.
To save time on subsequent runs, add the ```--skip-tags clone_mf_parsers``` to prevent repeating the downloading.

In [ ]:
echo "This will install Prommetheus Mpnitoring System to rack ---$RACK_TO_INSTALL---"

In [ ]:
# note if --tags then ony tagged sections will be executed:
# --skip-tags clone_mf_parsers  will skip tagged sections matching name, and execute all other sections
# other command line options:
# --check can be added to check the install without actually performing the install
#  --ask-vault-pass will ask for the password for each run
ansible-playbook -i tmp_deployment_files/fabric-hosts playbook_fabric_rack_prometheus_install.yml --extra-vars "rack_to_install=$RACK_TO_INSTALL"
